#FINRL Portfolio Allocation

#Install all the packages through FinRL library


In [1]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git


  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-mr3vi57q
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-mr3vi57q
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-faav1w2r/pyfolio_2a47c45dc2d0485c92321dc74c3698ce
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-faav1w2r/pyfolio_2a47c45dc2d0485c92321dc74c3698ce
     |████████████████████████████████| 172 kB 13.0 MB/s 
     |████████████████████████████████| 52 kB 2.1 MB/s 
     |████████████████████████████████| 6.3 MB 21.1 MB/s 
  Created wheel for finrl: filename=finrl-0.3.0-py3-none-any.whl size=69322 sha256=a58c5e35a45cb74ebc7c3bcf87562eb9db3483a2c69a90ca820fe8ff9505611d
  Stored in directory: /tmp/pip-ephem-wheel-cache-45lhsfbu/wheels/17/ff/bd/1bc602a0352762b0b24041b88536d803ae343ed0a711fcf55e
  Created wheel for pyfolio: filename=pyfolio-0.9.2+75.g4b901f6-py3-none-any.whl size=75

#Import Packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
import datetime

from finrl.apps import config
from finrl.neo_finrl.preprocessor.yahoodownloader import YahooDownloader
from finrl.neo_finrl.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.neo_finrl.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts


import sys
sys.path.append("../FinRL-Library")

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [3]:
import pandas as pd
import yfinance as yf

# Create Folders

In [4]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

#Download Data

In [5]:
x_change = "USDTRY=X" #x_change data downloaded
cx_data = yf.download(tickers=x_change , start="2015-07-27", end="2021-07-28" )

[*********************100%***********************]  1 of 1 completed


In [6]:
tickers = ["VESTL.IS","VAKBN.IS","TUPRS.IS","THYAO.IS","HALKB.IS","SASA.IS","PETKM.IS","SAHOL.IS","PGSUS.IS","GARAN.IS","AKBNK.IS","ARCLK.IS","BIMAS.IS","EREGL.IS"] # 20 tane hisse


In [7]:
date1='2015-07-27'
date2="2021-07-28"
list_data = []
for ticker in tickers:
  data = yf.download(tickers=ticker , start=date1, end=date2, interval='1d' )
  data["tic"] = ticker
  data["date"] = data.index
  data = data.reset_index()
  list_data.append(data)

import pandas as pd
df = pd.concat(list_data)

del df['Date']
df.isna().sum()
multi_ticker_df = df.ffill().bfill()
print(multi_ticker_df)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
           Open       High        

In [8]:
multi_ticker_df.columns= multi_ticker_df.columns.str.strip().str.lower()
multi_ticker_df.columns


Index(['open', 'high', 'low', 'close', 'adj close', 'volume', 'tic', 'date'], dtype='object')

In [9]:
def prefill(multi_ticker_df)  :
    df = multi_ticker_df.copy()
    df=df.sort_values(['date','tic'],ignore_index=True)
    df.index = df.date.factorize()[0]
    merged_closes = df.pivot_table(index = 'date',columns = 'tic', values = 'close')

    print(merged_closes.isna().sum())
    merged_closes = merged_closes.ffill().bfill()

    # print(merged_closes.isna().sum())
    # print(merged_closes)
    tics = merged_closes.columns
    df = df[df.tic.isin(tics)]
    df = df.reset_index()
    del df['index']
    return df
multiticker_df = prefill(multi_ticker_df)

tic
AKBNK.IS    0
ARCLK.IS    0
BIMAS.IS    0
EREGL.IS    0
GARAN.IS    0
HALKB.IS    0
PETKM.IS    0
PGSUS.IS    0
SAHOL.IS    0
SASA.IS     0
THYAO.IS    0
TUPRS.IS    0
VAKBN.IS    0
VESTL.IS    0
dtype: int64


#Preprocess Data

In [10]:

fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = True)

df = fe.preprocess_data(multiticker_df)



Successfully added technical indicators
Successfully added user defined features


In [11]:
df

,open,high,low,close,adj close,volume,tic,date,macd,rsi_30,cci_30,dx_30,daily_return
0,6.182181,6.182181,6.122160,6.165032,5.510608,13466375,AKBNK.IS,2015-07-27,0.000000,0.000000,66.666667,100.000000,1.368195
1531,14.750000,14.950000,14.500000,14.600000,11.878804,1976842,ARCLK.IS,2015-07-27,0.000000,0.000000,66.666667,100.000000,1.368195
3062,23.525000,23.799999,23.325001,23.600000,19.072001,628882,BIMAS.IS,2015-07-27,0.000000,0.000000,66.666667,100.000000,0.616438
4593,3.990000,4.040000,3.960000,4.040000,1.965522,6776403,EREGL.IS,2015-07-27,0.000000,0.000000,66.666667,100.000000,-0.828814
6124,7.900000,7.910000,7.740000,7.760000,6.942376,63095366,GARAN.IS,2015-07-27,0.000000,0.000000,66.666667,100.000000,0.920792
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15309,29.680000,30.299999,29.340000,29.540001,29.540001,15605997,SASA.IS,2021-07-27,-0.567082,47.213518,-26.532921,13.879146,2.179763
16840,12.310000,12.600000,12.240000,12.550000,12.550000,90420698,THYAO.IS,2021-07-27,-0.338376,42.154444,-164.845461,45.507105,-0.575152
18371,91.849998,92.500000,91.000000,92.349998,92.349998,2781736,TUPRS.IS,2021-07-27,-1.613261,42.913015,-125.319653,47.509144,6.358566
19902,3.430000,3.440000,3.390000,3.420000,3.420000,30871452,VAKBN.IS,2021-07-27,-0.036190,42.210935,-143.798826,35.035782,-0.962967


In [12]:
df.head()


,open,high,low,close,adj close,volume,tic,date,macd,rsi_30,cci_30,dx_30,daily_return
0,6.182181,6.182181,6.122160,6.165032,5.510608,13466375,AKBNK.IS,2015-07-27,0.0,0.0,66.666667,100.0,1.368195
1531,14.750000,14.950000,14.500000,14.600000,11.878804,1976842,ARCLK.IS,2015-07-27,0.0,0.0,66.666667,100.0,1.368195
3062,23.525000,23.799999,23.325001,23.600000,19.072001,628882,BIMAS.IS,2015-07-27,0.0,0.0,66.666667,100.0,0.616438
4593,3.990000,4.040000,3.960000,4.040000,1.965522,6776403,EREGL.IS,2015-07-27,0.0,0.0,66.666667,100.0,-0.828814
6124,7.900000,7.910000,7.740000,7.760000,6.942376,63095366,GARAN.IS,2015-07-27,0.0,0.0,66.666667,100.0,0.920792


##Add covariance matrix as states

In [13]:

# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values 
  cov_list.append(covs)

  
df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)

In [14]:
df.shape


(17906, 15)

In [15]:
df.head()


,open,high,low,close,adj close,volume,tic,date,macd,rsi_30,cci_30,dx_30,daily_return,cov_list,return_list
0,7.536945,7.545519,6.919584,7.048201,6.421494,68604764,AKBNK.IS,2016-07-18,0.150507,51.079179,43.355083,16.994584,-0.000255,"[[0.0003865908895953564, 0.0002229728849683487...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...
1,20.420000,21.000000,19.900000,20.320000,16.874163,1910358,ARCLK.IS,2016-07-18,0.283658,53.487901,50.036902,7.787718,1.883005,"[[0.0003865908895953564, 0.0002229728849683487...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...
2,28.549999,29.125000,27.250000,27.375000,22.502356,4072774,BIMAS.IS,2016-07-18,-0.112787,42.782696,-95.904875,24.633324,0.347195,"[[0.0003865908895953564, 0.0002229728849683487...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...
3,4.620000,4.690000,4.480000,4.690000,2.448715,22702083,EREGL.IS,2016-07-18,0.127986,60.239940,198.596835,21.990918,-0.828676,"[[0.0003865908895953564, 0.0002229728849683487...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...
4,8.320000,8.320000,7.400000,7.680000,6.982357,257035312,GARAN.IS,2016-07-18,0.130669,48.689893,24.682251,19.222105,0.637527,"[[0.0003865908895953564, 0.0002229728849683487...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...


In [16]:
# x_change data preprocessed and only ad close column stay with date column
cx_data.columns= cx_data.columns.str.strip().str.lower()
del cx_data["open"]
del cx_data["high"]
del cx_data["low"]
del cx_data["volume"]
del cx_data["close"]
cx_data["x_change"] = cx_data["adj close"]
del cx_data["adj close"]

cx_data = cx_data.reset_index()
cx_data.columns= cx_data.columns.str.strip().str.lower()

cx_data

,date,x_change
0,2015-07-27,2.74419
1,2015-07-28,2.76818
2,2015-07-29,2.75730
3,2015-07-30,2.76576
4,2015-07-31,2.78766
...,...,...
1540,2021-07-22,8.55416
1541,2021-07-23,8.55650
1542,2021-07-26,8.54757
1543,2021-07-27,8.55450


In [17]:
#X_change df and preprocessed df merged based on "date" column
merged_df = pd.merge(df, cx_data, on='date')

In [18]:
df = merged_df
df #final df

,open,high,low,close,adj close,volume,tic,date,macd,rsi_30,cci_30,dx_30,daily_return,cov_list,return_list,x_change
0,7.536945,7.545519,6.919584,7.048201,6.421494,68604764,AKBNK.IS,2016-07-18,0.150507,51.079179,43.355083,16.994584,-0.000255,"[[0.0003865908895953564, 0.0002229728849683487...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...,2.96626
1,20.420000,21.000000,19.900000,20.320000,16.874163,1910358,ARCLK.IS,2016-07-18,0.283658,53.487901,50.036902,7.787718,1.883005,"[[0.0003865908895953564, 0.0002229728849683487...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...,2.96626
2,28.549999,29.125000,27.250000,27.375000,22.502356,4072774,BIMAS.IS,2016-07-18,-0.112787,42.782696,-95.904875,24.633324,0.347195,"[[0.0003865908895953564, 0.0002229728849683487...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...,2.96626
3,4.620000,4.690000,4.480000,4.690000,2.448715,22702083,EREGL.IS,2016-07-18,0.127986,60.239940,198.596835,21.990918,-0.828676,"[[0.0003865908895953564, 0.0002229728849683487...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...,2.96626
4,8.320000,8.320000,7.400000,7.680000,6.982357,257035312,GARAN.IS,2016-07-18,0.130669,48.689893,24.682251,19.222105,0.637527,"[[0.0003865908895953564, 0.0002229728849683487...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...,2.96626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17579,29.680000,30.299999,29.340000,29.540001,29.540001,15605997,SASA.IS,2021-07-27,-0.567082,47.213518,-26.532921,13.879146,2.179763,"[[0.0005066583324306413, 0.0001591826865959864...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...,8.55450
17580,12.310000,12.600000,12.240000,12.550000,12.550000,90420698,THYAO.IS,2021-07-27,-0.338376,42.154444,-164.845461,45.507105,-0.575152,"[[0.0005066583324306413, 0.0001591826865959864...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...,8.55450
17581,91.849998,92.500000,91.000000,92.349998,92.349998,2781736,TUPRS.IS,2021-07-27,-1.613261,42.913015,-125.319653,47.509144,6.358566,"[[0.0005066583324306413, 0.0001591826865959864...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...,8.55450
17582,3.430000,3.440000,3.390000,3.420000,3.420000,30871452,VAKBN.IS,2021-07-27,-0.036190,42.210935,-143.798826,35.035782,-0.962967,"[[0.0005066583324306413, 0.0001591826865959864...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...,8.55450


#Design Environment

In [19]:
train = data_split(df, '2015-07-27','2020-07-27')


In [20]:
train.head()


,open,high,low,close,adj close,volume,tic,date,macd,rsi_30,cci_30,dx_30,daily_return,cov_list,return_list,x_change
0,7.536945,7.545519,6.919584,7.048201,6.421494,68604764,AKBNK.IS,2016-07-18,0.150507,51.079179,43.355083,16.994584,-0.000255,"[[0.0003865908895953564, 0.0002229728849683487...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...,2.96626
0,20.420000,21.000000,19.900000,20.320000,16.874163,1910358,ARCLK.IS,2016-07-18,0.283658,53.487901,50.036902,7.787718,1.883005,"[[0.0003865908895953564, 0.0002229728849683487...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...,2.96626
0,28.549999,29.125000,27.250000,27.375000,22.502356,4072774,BIMAS.IS,2016-07-18,-0.112787,42.782696,-95.904875,24.633324,0.347195,"[[0.0003865908895953564, 0.0002229728849683487...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...,2.96626
0,4.620000,4.690000,4.480000,4.690000,2.448715,22702083,EREGL.IS,2016-07-18,0.127986,60.239940,198.596835,21.990918,-0.828676,"[[0.0003865908895953564, 0.0002229728849683487...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...,2.96626
0,8.320000,8.320000,7.400000,7.680000,6.982357,257035312,GARAN.IS,2016-07-18,0.130669,48.689893,24.682251,19.222105,0.637527,"[[0.0003865908895953564, 0.0002229728849683487...",tic AKBNK.IS ARCLK.IS BIMAS.IS ... ...,2.96626


#Environment for Portfolio Allocation

In [21]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,)) 
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold        
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()
            
            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))           
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions) 
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            #print(self.state)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])            
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value 
            #print("Step reward: ", self.reward)
            #self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [22]:
config.TECHNICAL_INDICATORS_LIST.append("x_change") # x_change column added to indicator list . So ENV can see the new column.
config.TECHNICAL_INDICATORS_LIST

['macd', 'rsi_30', 'cci_30', 'dx_30', 'x_change']

In [23]:
print(config.TECHNICAL_INDICATORS_LIST)

['macd', 'rsi_30', 'cci_30', 'dx_30', 'x_change']


In [24]:

stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 14, State Space: 14


In [25]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

In [26]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


#Implement DRL ALgorithms
##Model 1: A2C

In [27]:
# initialize
agent = DRLAgent(env = env_train)

In [28]:
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cuda device


In [29]:
trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=50000)

Logging to tensorboard_log/a2c/a2c_1
------------------------------------
| time/                 |          |
|    fps                | 159      |
|    iterations         | 100      |
|    time_elapsed       | 3        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -19.8    |
|    explained_variance | 1.19e-07 |
|    learning_rate      | 0.0002   |
|    n_updates          | 99       |
|    policy_loss        | 7.49e+07 |
|    std                | 0.999    |
|    value_loss         | 2.25e+13 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 216      |
|    iterations         | 200      |
|    time_elapsed       | 4        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -19.8    |
|    explained_variance | 2.98e-07 |
|    learning_rate      | 0.0002   |
|    n_updates          | 199      |
|

#Trading

In [30]:
trade = data_split(df,'2020-07-27', '2021-07-28')
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)

In [31]:
trade.shape


(3486, 16)

In [32]:
df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

begin_total_asset:1000000
end_total_asset:1273247.4292903307
Sharpe:  1.0960822936307981
hit end!


In [33]:
df_daily_return.head()


,date,daily_return
0,2020-07-27,0.000000
1,2020-07-28,-0.033854
2,2020-07-29,-0.020393
3,2020-07-30,-0.008348
4,2020-08-04,-0.025086


In [34]:
df_daily_return.to_csv('df_daily_return.csv')


In [35]:
df_actions.head()


,AKBNK.IS,ARCLK.IS,BIMAS.IS,EREGL.IS,GARAN.IS,HALKB.IS,PETKM.IS,PGSUS.IS,SAHOL.IS,SASA.IS,THYAO.IS,TUPRS.IS,VAKBN.IS,VESTL.IS
date,,,,,,,,,,,,,,
2020-07-27,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429
2020-07-28,0.052789,0.121733,0.044783,0.044783,0.069624,0.046928,0.121733,0.044783,0.044783,0.075767,0.121733,0.044783,0.061687,0.104092
2020-07-29,0.053592,0.053592,0.145679,0.053592,0.053592,0.053592,0.117840,0.118101,0.053592,0.082457,0.053592,0.053592,0.053592,0.053592
2020-07-30,0.046265,0.046265,0.063075,0.125762,0.120000,0.046265,0.046265,0.125762,0.069515,0.046265,0.046265,0.046265,0.125762,0.046265
2020-08-04,0.050515,0.099936,0.050515,0.050515,0.137314,0.050515,0.084679,0.050515,0.050515,0.076000,0.115845,0.050515,0.082105,0.050515


In [36]:
df_actions.to_csv('df_actions.csv')


#Backtest Our Strategy
##BackTestStats

In [37]:
from pyfolio import timeseries
DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
perf_func = timeseries.perf_stats 
perf_stats_all = perf_func( returns=DRL_strat, 
                              factor_returns=DRL_strat, 
                                positions=None, transactions=None, turnover_denom="AGB")

In [38]:

print("==============DRL Strategy Stats===========")
perf_stats_all

==============DRL Strategy Stats===========


Annual return          0.276959
Cumulative returns     0.273247
Annual volatility      0.252414
Sharpe ratio           1.096082
Calmar ratio           1.842318
Stability              0.736676
Max drawdown          -0.150332
Omega ratio            1.219001
Sortino ratio          1.515389
Skew                  -1.004108
Kurtosis               6.158347
Tail ratio             1.045370
Daily value at risk   -0.030703
Alpha                  0.000000
Beta                   1.000000
dtype: float64

In [39]:

#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_daily_return.loc[0,'date'],
        end = df_daily_return.loc[len(df_daily_return)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')

==============Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (252, 8)
Annual return          0.321972
Cumulative returns     0.321972
Annual volatility      0.143690
Sharpe ratio           2.023038
Calmar ratio           3.605191
Stability              0.938334
Max drawdown          -0.089308
Omega ratio            1.402453
Sortino ratio          2.993793
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.072468
Daily value at risk   -0.016950
dtype: float64


## BackTestPlot

In [42]:

# import pyfolio
# %matplotlib inline

# baseline_df = get_baseline(
#         ticker='^DJI', start=df_daily_return.loc[0,'date'], end='2021-07-01'
#     )

# baseline_returns = get_daily_return(baseline_df, value_col_name="close")

# with pyfolio.plotting.plotting_context(font_scale=1.1):
#         pyfolio.create_full_tear_sheet(returns = DRL_strat,
#                                        benchmark_rets=baseline_returns, set_context=False)